# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Project 2 - Singapore Housing Data and Kaggle Challenge

# Part 3

## 1) Importing Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge, RidgeCV
import pickle

# Set the option to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
train = pd.read_csv("data/train_cleaned.csv", low_memory = False)
test = pd.read_csv("data/test.csv", low_memory = False)
sample = pd.read_csv("data/sample_sub_reg.csv", low_memory = False)
num_index = ['hdb_age', 'floor_area_sqm', 'max_floor_lvl', 'sec_sch_nearest_dist']
cat_index = ['flat_type', 'storey_range', 'flat_model', 'full_flat_type', 'region']

In [3]:
# How our final result should look like
print(sample.shape)
sample.head()

(16737, 2)


,Id,Predicted
0,114982,500000
1,95653,500000
2,40303,500000
3,109506,500000
4,100149,500000


In [4]:
# Checking test.csv
print(test.shape)
test.head()

(16737, 76)


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,full_flat_type,address,floor_area_sqft,hdb_age,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,postal,Latitude,Longitude,planning_area,Mall_Nearest_Distance,Mall_Within_500m,Mall_Within_1km,Mall_Within_2km,Hawker_Nearest_Distance,Hawker_Within_500m,Hawker_Within_1km,Hawker_Within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_name,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,114982,2012-11,YISHUN,4 ROOM,173,YISHUN AVE 7,07 TO 09,84.0,Simplified,1987,2012,11,8,7,9,8,4 ROOM Simplified,"173, YISHUN AVE 7",904.176,34,12,1986,Y,Y,N,N,N,132,0,0,0,92,40,0,0,0,0,0,0,0,760173,1.437066,103.831121,Yishun,877.431572,NaN,2.0,4.0,687.576779,NaN,1.0,1.0,56,123,686.660434,Canberra,0,0,1.443077,103.829703,75.683952,Blk 174,1.437558,103.831591,426.467910,Ahmad Ibrahim Primary School,92,0,1.433681,103.832924,156.322353,Ahmad Ibrahim Secondary School,218,0,1.436235,103.829987
1,95653,2019-08,JURONG WEST,5 ROOM,986C,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,2008,2019,8,5,4,6,5,5 ROOM Premium Apartment,"986C, JURONG WEST ST 93",1205.568,13,14,2002,Y,N,N,N,N,53,0,0,0,28,25,0,0,0,0,0,0,0,643986,1.336957,103.695668,Jurong West,534.037705,NaN,1.0,3.0,2122.346226,NaN,NaN,NaN,72,94,169.478175,Pioneer,0,0,1.337343,103.697143,88.993058,Blk 653B,1.336491,103.696319,439.756851,Jurong West Primary School,45,0,1.339244,103.698896,739.371688,Jurong West Secondary School,199,0,1.335256,103.702098
2,40303,2013-10,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,2013,10,8,7,9,8,3 ROOM New Generation,"534, ANG MO KIO AVE 10",731.952,41,12,1979,Y,N,N,N,N,218,0,0,191,22,1,1,0,0,0,0,3,0,560534,1.374058,103.854168,Ang Mo Kio,817.050453,NaN,2.0,3.0,152.287621,1.0,3.0,11.0,50,100,694.220448,Ang Mo Kio,1,0,1.369465,103.849939,86.303575,Blk 532,1.374255,103.854919,355.882207,Jing Shan Primary School,36,0,1.371893,103.851811,305.071191,Anderson Secondary School,245,0,1.374242,103.851430
3,109506,2017-10,WOODLANDS,4 ROOM,29,MARSILING DR,01 TO 03,97.0,New Generation,1979,2017,10,2,1,3,2,4 ROOM New Generation,"29, MARSILING DR",1044.108,42,14,1976,Y,N,N,N,N,104,0,0,0,104,0,0,0,0,0,0,0,0,731029,1.442748,103.772922,Woodlands,1272.737194,NaN,NaN,3.0,501.892158,NaN,1.0,2.0,52,112,1117.203587,Marsiling,0,0,1.432757,103.773982,108.459039,Blk 32,1.443650,103.773295,929.744711,Marsiling Primary School,54,0,1.434423,103.773698,433.454591,Woodlands Secondary School,188,0,1.439183,103.774499
4,100149,2016-08,BUKIT BATOK,4 ROOM,170,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,1985,2016,8,17,16,18,17,4 ROOM Model A,"170, BT BATOK WEST AVE 8",1108.692,36,25,1985,Y,N,N,N,N,144,0,0,0,48,96,0,0,0,0,0,0,0,650170,1.346556,103.740101,Bukit Batok,1070.963675,NaN,NaN,5.0,437.593564,1.0,2.0,2.0,60,87,987.976010,Chinese Garden,0,0,1.342441,103.732225,113.645431,Blk 169,1.346899,103.741064,309.926934,Princess Elizabeth Primary School,40,0,1.349195,103.741000,217.295361,Bukit Batok Secondary School,223,0,1.348351,103.740873


## 2) Predictions on Test/Holdout Set

### Using Part 1's Data Cleaning on test.csv

In [5]:
test.columns = [x.lower().replace(' ','_') for x in test.columns]

# Renaming columns for better differentiation
test.rename(columns={'affiliation': 'sec_sch_affiliation'}, inplace=True)

# Dropping columns with NaN and columns with no relavant info
columns_to_drop = ['block', 'street_name', 'mid_storey', 'address', 'postal', 'latitude', 'longitude', 
                   'planning_area', 'mrt_latitude', 'mrt_longitude', 'bus_stop_name', 'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_latitude', 
                   'pri_sch_longitude', 'sec_sch_latitude', 'sec_sch_longitude', 'mall_within_500m', 'mall_within_1km', 'mall_within_2km', 
                   'hawker_within_500m', 'hawker_within_1km', 'hawker_within_2km', 'tranc_yearmonth', 'mrt_name', 'pri_sch_name', 'sec_sch_name', 
                   'floor_area_sqft', 'lower', 'mid', 'upper', 'bus_stop_nearest_distance', 'residential', 'other_room_rental', 
                   '3room_rental', '2room_rental', '1room_rental', 'studio_apartment_sold', 'multigen_sold', 'exec_sold', '5room_sold', '4room_sold', 
                   '3room_sold', '2room_sold', '1room_sold', 'lease_commence_date']  

test = test.drop(columns=columns_to_drop, axis=1)

# Convert all NaN values of mall_nearest_distance with mean values of mall_nearest_distance
mean_distance = test['mall_nearest_distance'].mean()
test['mall_nearest_distance'].fillna(mean_distance, inplace=True)
test.head()

# Grouping town into regions and creating a new column ['region']
area_to_region = {
    'KALLANG/WHAMPOA': 'Central',
    'BISHAN': 'Central',
    'BUKIT BATOK': 'West',
    'YISHUN': 'North',
    'GEYLANG': 'Central',
    'HOUGANG': 'North-East',
    'BEDOK': 'East',
    'SENGKANG': 'North-East',
    'TAMPINES': 'East',
    'SERANGOON': 'North-East',
    'BUKIT MERAH': 'Central',
    'BUKIT PANJANG': 'West',
    'WOODLANDS': 'North',
    'JURONG WEST': 'West',
    'TOA PAYOH': 'Central',
    'CHOA CHU KANG': 'West',
    'SEMBAWANG': 'North',
    'ANG MO KIO': 'North-East',
    'PASIR RIS': 'East',
    'CLEMENTI': 'West',
    'PUNGGOL': 'North-East',
    'JURONG EAST': 'West',
    'CENTRAL AREA': 'Central',
    'QUEENSTOWN': 'Central',
    'BUKIT TIMAH': 'Central',
    'MARINE PARADE': 'East'
}

test['region'] = test['town'].map(area_to_region)
test.drop(columns='town', axis=1, inplace=True)

columns_to_convert = ['commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion']

for column in columns_to_convert:
    test[column] = test[column].map({'Y': 1, 'N': 0})

test_floor_mapping = {
    '01 TO 03': 'low',
    '01 TO 05': 'low',
    '04 TO 06': 'low',
    '06 TO 10': 'low',
    '07 TO 09': 'low',
    '10 TO 12': 'mid',
    '11 TO 15': 'mid',
    '13 TO 15': 'mid',
    '16 TO 18': 'mid',
    '16 TO 20': 'mid',
    '19 TO 21': 'mid',
    '21 TO 25': 'high',
    '22 TO 24': 'high',
    '25 TO 27': 'high',
    '26 TO 30': 'high',
    '28 TO 30': 'high',
    '31 TO 33': 'veryhigh',
    '31 TO 35': 'veryhigh',
    '34 TO 36': 'veryhigh',
    '36 TO 40': 'veryhigh',
    '37 TO 39': 'veryhigh',
    '40 TO 42': 'ultrahigh',
    '43 TO 45': 'ultrahigh',
    '46 TO 48': 'ultrahigh',
    '49 TO 51': 'ultrahigh'
}

test['storey_range'] = [test_floor_mapping[value] for value in test['storey_range']]

### Using Part 2's Modeling on test.csv

In [6]:
# Load the model from the file
with open('ridgecv_model.pkl', 'rb') as file:
    ridge = pickle.load(file)

In [7]:
# Load the model from the file
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [8]:
# Identify numerical and categorical features in your data (replace with your actual column names)
numeric_features = num_index
categorical_features = cat_index

In [9]:
# One hot encode the categorical features using pd.get_dummies
X_train_encoded = pd.get_dummies(train[categorical_features], drop_first=True)
test_encoded = pd.get_dummies(test[categorical_features], drop_first=True)

# Reindex the testing set to match the columns in the training set
test_encoded = test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Apply StandardScaler to the numerical features to test
test_encoded[numeric_features] = scaler.transform(test[numeric_features])

In [10]:
print(test_encoded.shape)
test_encoded.head()

(16737, 79)


,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION,storey_range_low,storey_range_mid,storey_range_ultrahigh,storey_range_veryhigh,flat_model_Adjoined flat,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,flat_model_Improved-Maisonette,flat_model_Maisonette,flat_model_Model A,flat_model_Model A-Maisonette,flat_model_Model A2,flat_model_Multi Generation,flat_model_New Generation,flat_model_Premium Apartment,flat_model_Premium Apartment Loft,flat_model_Premium Maisonette,flat_model_Simplified,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2,full_flat_type_2 ROOM 2-room,full_flat_type_2 ROOM DBSS,full_flat_type_2 ROOM Improved,full_flat_type_2 ROOM Model A,full_flat_type_2 ROOM Premium Apartment,full_flat_type_2 ROOM Standard,full_flat_type_3 ROOM DBSS,full_flat_type_3 ROOM Improved,full_flat_type_3 ROOM Model A,full_flat_type_3 ROOM New Generation,full_flat_type_3 ROOM Premium Apartment,full_flat_type_3 ROOM Simplified,full_flat_type_3 ROOM Standard,full_flat_type_3 ROOM Terrace,full_flat_type_4 ROOM Adjoined flat,full_flat_type_4 ROOM DBSS,full_flat_type_4 ROOM Improved,full_flat_type_4 ROOM Model A,full_flat_type_4 ROOM Model A2,full_flat_type_4 ROOM New Generation,full_flat_type_4 ROOM Premium Apartment,full_flat_type_4 ROOM Premium Apartment Loft,full_flat_type_4 ROOM Simplified,full_flat_type_4 ROOM Standard,full_flat_type_4 ROOM Terrace,full_flat_type_4 ROOM Type S1,full_flat_type_5 ROOM Adjoined flat,full_flat_type_5 ROOM DBSS,full_flat_type_5 ROOM Improved,full_flat_type_5 ROOM Improved-Maisonette,full_flat_type_5 ROOM Model A,full_flat_type_5 ROOM Model A-Maisonette,full_flat_type_5 ROOM Premium Apartment,full_flat_type_5 ROOM Premium Apartment Loft,full_flat_type_5 ROOM Standard,full_flat_type_5 ROOM Type S2,full_flat_type_EXECUTIVE Adjoined flat,full_flat_type_EXECUTIVE Apartment,full_flat_type_EXECUTIVE Maisonette,full_flat_type_EXECUTIVE Premium Apartment,full_flat_type_EXECUTIVE Premium Maisonette,full_flat_type_MULTI-GENERATION Multi Generation,region_East,region_North,region_North-East,region_West,hdb_age,floor_area_sqm,max_floor_lvl,sec_sch_nearest_dist
0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.451495,-0.544595,-0.503336,-1.139871
1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,-1.280213,0.603098,-0.183015,0.743739
2,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1.028731,-1.200419,-0.503336,-0.659320
3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.111193,-0.011737,-0.183015,-0.244563
4,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.616419,0.234197,1.578746,-0.942891


In [11]:
# Predict the test results using our model
test_prediction = ridge.predict(test_encoded)

In [12]:
# Create a dataframe for our predictions
result = pd.DataFrame(test['id'])

# Rename the column as per the sample file
result.rename(columns={'id': 'Id'}, inplace=True)

# Include our predictions in the result df
result['Predicted'] = test_prediction

In [13]:
# Check the first 5 rows of our result df
print(result.shape)
result.head()

(16737, 2)


,Id,Predicted
0,114982,323767.839384
1,95653,454366.375735
2,40303,310165.773913
3,109506,376965.221039
4,100149,457949.070171


## 3) Kaggle Submission

In [14]:
# Save predictions as result.csv to be submitted to Kaggle
result.to_csv('For Submission/result.csv', index=False)

![screenshot](images/Screenshot1.png)

<u>**Kaggle result**</u>:

- Score: <u>66092.88454</u>
- Private score: <u>66593.2626</u>

## 4) Conclusion and Recommendations

### Findings

**Top Predictors** 
 - Flat models
 - Regions
 - Floor area sqm

**Conclusions**:

- Predictive Accuracy: The ridge regression model has shown promising results in accurately predicting housing prices in Singapore based on various factors, including property features, location, economic indicators, and demographic trends. Comparing the results of the test and train set, there is a margin of error at **<u>0.99%</u>**. 

- Significant Factors: The model has identified the key drivers of soaring housing prices, allowing policymakers to gain insights into the most influential factors contributing to the increase. From the model, we can safely assume that some features of the model like `Flat models`, `Regions` and the `Floor area sqm` are some of the strong key drivers of the housing price.

- Model Robustness: The ridge regression model's regularization parameter (alpha) has been tuned to find an optimal value, ensuring the model's stability and reducing the risk of overfitting.

- Policy Insights: The regression model's outputs have provided valuable insights into the relationships between different variables and housing prices, guiding policymakers in formulating evidence-based policies.

**Recommendations**:

- Address Supply Constraints: Based on the model's findings, policymakers should focus on increasing the supply of housing units to meet the growing demand. This could involve optimizing land use, redeveloping underutilized areas, and streamlining approval processes for new housing developments.

- Targeted Affordable Housing Initiatives: Utilize the regression model to identify areas with the greatest affordability challenges and design targeted affordable housing initiatives. These initiatives should cater to the needs of low and middle-income families, providing them with access to more affordable housing options.

- Sustainable Urban Planning: Implement sustainable urban planning practices that promote balanced growth and ensure housing affordability. This includes integrating residential, commercial, and recreational spaces to create vibrant and self-sustaining communities.

- Manage Speculation: To curb excessive speculation and stabilize the housing market, consider implementing measures such as property cooling measures or capital gains taxes on property transactions.

- Public-Private Partnerships: Foster collaborations between the public and private sectors to address housing affordability issues. Encourage developers to participate in affordable housing projects and provide incentives for creating housing units that cater to diverse income groups.

- Regular Market Monitoring: Continuously monitor the real estate market and economic indicators to detect potential bubbles or price fluctuations. Implement timely interventions to mitigate risks and maintain market stability.

- Data-Driven Policy Decisions: Leverage the predictive capabilities of the ridge regression model to make informed policy decisions. Regularly update and refine the model using new data to ensure its relevance and accuracy.

- Community Engagement: Involve residents and stakeholders in the policy development process to understand their housing needs and preferences. Engaging the community will lead to more inclusive and effective housing policies.

By adopting these recommendations and leveraging the insights from the ridge regression model, policymakers can work towards a more sustainable and balanced real estate market in Singapore, ensuring that housing remains affordable for the residents while fostering responsible growth and development in the housing sector.

## 5) Moving Forward

While the predictive model and key features provide great insights for the investor, we are still limited to the data quality and the limited features. Regression models assume a linear relationship between the independent variables and the dependent variable. If the true relationship is non-linear, the model may not accurately capture the underlying patterns in the data. The model's predictions may also not be reliable when making predictions beyond the range of the training data (extrapolation). The model might not have enough information to accurately predict prices in unobserved scenarios.

To enchance the model and improve performance, we can consider the following:

- Include external factors like government policies, interests rates and macroeconomic indicators that may impact housing prices.
- Using other more advance forms of regression models.
- Collaborate closely with stakeholders, policymakers, and domain experts throughout the model development process. Gather feedback and insights from them to ensure that the model's recommendations align with their needs and priorities.

![housing](images/future-housing.jpg)